# torch.nn.parameter.Parameter

## Тодорхойлолт
- **Parameter** нь PyTorch-ийн Tensor-ийн дэд анги бөгөөд модулийн сургагдах параметр болохыг илэрхийлдэг. Энэ нь автоматаар градиент тооцоонд орж, суралцах процессод оролцдог.

## Үндсэн онцлог

`torch.nn.parameter.Parameter(data=None, requires_grad=True)`

##Аргументууд:

- **data (Tensor, optional):** Parameter-ын утга

- **requires_grad (bool, optional):** Градиент тооцоолол хийх эсэх. Анхны утга: True

## Үндсэн ялгаа

### Parameter vs Tensor

In [1]:
import torch
import torch.nn as nn

# Энгийн Tensor
tensor = torch.tensor([1.0, 2.0, 3.0])
print(f"Энгийн Tensor: {type(tensor)}")
print(f"requires_grad: {tensor.requires_grad}")
print(f"Параметр мөн үү? {isinstance(tensor, nn.Parameter)}")

# Parameter
parameter = nn.Parameter(torch.tensor([1.0, 2.0, 3.0]))
print(f"\nParameter: {type(parameter)}")
print(f"requires_grad: {parameter.requires_grad}")
print(f"Параметр мөн үү? {isinstance(parameter, nn.Parameter)}")

Энгийн Tensor: <class 'torch.Tensor'>
requires_grad: False
Параметр мөн үү? False

Parameter: <class 'torch.nn.parameter.Parameter'>
requires_grad: True
Параметр мөн үү? True


## nn.Module доторх ажиллагаа

In [2]:
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Tensor болгон хадгалсан (буруу)
        self.weight_tensor = torch.randn(3, 3)  # Алдаа: параметр болохгүй
        
        # Parameter болгон хадгалсан (зөв)
        self.weight_param = nn.Parameter(torch.randn(3, 3))
        
        # Module дотор Parameter үүсгэсэн
        self.linear = nn.Linear(3, 2)  # Дотор нь Parameter-ууд агуулна
    
    def forward(self, x):
        return torch.matmul(x, self.weight_param)  # Зөвхөн parameter ажиллана

# Моделийг үүсгэх
model = SimpleModel()

# Бүх параметрүүдийг харах
print("Бүх параметрүүд:")
for name, param in model.named_parameters():
    print(f"  {name}: {param.shape}")

# Зөвхөн parameter л нэрсийн жагсаалтад гарна
print(f"\nweight_tensor параметр мөн үү? {'weight_tensor' in dict(model.named_parameters())}")
print(f"weight_param параметр мөн үү? {'weight_param' in dict(model.named_parameters())}")

Бүх параметрүүд:
  weight_param: torch.Size([3, 3])
  linear.weight: torch.Size([2, 3])
  linear.bias: torch.Size([2])

weight_tensor параметр мөн үү? False
weight_param параметр мөн үү? True


## Практик хэрэглээ

### 1. Гарын авлагын параметр үүсгэх

In [3]:
class ManualLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        
        # Жингийн матрицыг Parameter болгон үүсгэх
        self.weight = nn.Parameter(
            torch.randn(out_features, in_features) * 0.01  # Жижиг эхлэх утга
        )
        
        # Хазайлтыг Parameter болгон үүсгэх
        self.bias = nn.Parameter(torch.zeros(out_features))
    
    def forward(self, x):
        # Шугаман хувиргалт
        return torch.matmul(x, self.weight.t()) + self.bias

# Туршилт
manual_layer = ManualLinear(5, 3)
x = torch.randn(2, 5)
output = manual_layer(x)
print(f"Гаралтын хэлбэр: {output.shape}")
print(f"Параметрүүд: {list(manual_layer.named_parameters())}")

Гаралтын хэлбэр: torch.Size([2, 3])
Параметрүүд: [('weight', Parameter containing:
tensor([[-0.0103,  0.0019,  0.0062,  0.0073,  0.0002],
        [-0.0088, -0.0122,  0.0045,  0.0088, -0.0016],
        [-0.0027, -0.0101, -0.0111,  0.0001, -0.0018]], requires_grad=True)), ('bias', Parameter containing:
tensor([0., 0., 0.], requires_grad=True))]


### 2. Параметрүүдийг эхлүүлэх

In [4]:
import torch.nn.init as init

class InitializedModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Хэвийн тархалтаар эхлүүлэх
        self.param1 = nn.Parameter(torch.empty(3, 4))
        init.normal_(self.param1, mean=0, std=0.01)
        
        # Uniform тархалтаар эхлүүлэх
        self.param2 = nn.Parameter(torch.empty(2, 3))
        init.uniform_(self.param2, a=-0.1, b=0.1)
        
        # Constant утгаар эхлүүлэх
        self.param3 = nn.Parameter(torch.empty(5))
        init.constant_(self.param3, 0.5)
        
        # Xavier/Glorot эхлүүлэх
        self.param4 = nn.Parameter(torch.empty(6, 8))
        init.xavier_normal_(self.param4)
        
        # Kaiming/He эхлүүлэх (ReLU-тай сайн)
        self.param5 = nn.Parameter(torch.empty(10, 7))
        init.kaiming_uniform_(self.param5, mode='fan_in', nonlinearity='relu')

model = InitializedModel()

# Параметрүүдийн статистик харах
for name, param in model.named_parameters():
    print(f"{name}: дундаж={param.mean():.4f}, стандарт хазайлт={param.std():.4f}")

param1: дундаж=0.0029, стандарт хазайлт=0.0126
param2: дундаж=-0.0081, стандарт хазайлт=0.0627
param3: дундаж=0.5000, стандарт хазайлт=0.0000
param4: дундаж=-0.0139, стандарт хазайлт=0.3085
param5: дундаж=-0.0181, стандарт хазайлт=0.5487


### 3. Олон хэмжээст параметр

In [5]:
class MultiDimParameters(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Янз бүрийн хэмжээст параметрүүд
        self.scalar_param = nn.Parameter(torch.tensor(1.0))
        self.vector_param = nn.Parameter(torch.randn(10))
        self.matrix_param = nn.Parameter(torch.randn(5, 3))
        self.tensor_3d = nn.Parameter(torch.randn(2, 4, 6))
        self.tensor_4d = nn.Parameter(torch.randn(1, 3, 5, 7))
    
    def forward(self):
        # Параметрүүдийг ашиглах
        result = self.scalar_param * self.vector_param.sum()
        result += torch.sum(self.matrix_param)
        result += torch.mean(self.tensor_3d)
        result += torch.prod(self.tensor_4d)
        return result

model = MultiDimParameters()
output = model()
print(f"Гаралт: {output.item()}")
print(f"\nПараметрүүдийн хэмжээсүүд:")
for name, param in model.named_parameters():
    print(f"  {name}: {param.shape}")

Гаралт: 3.2150444984436035

Параметрүүдийн хэмжээсүүд:
  scalar_param: torch.Size([])
  vector_param: torch.Size([10])
  matrix_param: torch.Size([5, 3])
  tensor_3d: torch.Size([2, 4, 6])
  tensor_4d: torch.Size([1, 3, 5, 7])


### 4. requires_grad удирдлага

In [6]:
class GradControlModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Градиент тооцоолох
        self.trainable_param = nn.Parameter(
            torch.randn(3, 3), 
            requires_grad=True
        )
        
        # Градиент тооцохгүй (зөвхөн утга)
        self.fixed_param = nn.Parameter(
            torch.eye(3), 
            requires_grad=False
        )
        
        # Шаталт параметр
        self.temperature = nn.Parameter(
            torch.tensor(1.0), 
            requires_grad=False  # Ихэвчлэн градиент тооцохгүй
        )
    
    def forward(self, x):
        return torch.matmul(x, self.trainable_param * self.fixed_param) * self.temperature

model = GradControlModel()

# Суралцах боломжтой параметрүүд
trainable_params = []
for name, param in model.named_parameters():
    if param.requires_grad:
        trainable_params.append(name)

print(f"Суралцах боломжтой параметрүүд: {trainable_params}")
print(f"Нийт параметрүүдийн тоо: {len(list(model.parameters()))}")

# requires_grad өөрчлөх
model.fixed_param.requires_grad = True  # Одоо суралцах боломжтой
print(f"\nfixed_param одоо суралцах боломжтой үү? {model.fixed_param.requires_grad}")

Суралцах боломжтой параметрүүд: ['trainable_param']
Нийт параметрүүдийн тоо: 3

fixed_param одоо суралцах боломжтой үү? True


## Дэвшилтэт хэрэглээ
### 1. Параметр хавтгайруулах (flatten)

In [7]:
class FlattenedParameters(nn.Module):
    def __init__(self, total_params):
        super().__init__()
        # Бүх параметрүүдийг нэг векторт хадгалах
        self.flat_params = nn.Parameter(torch.randn(total_params))
        
        # Параметрүүдийг задлах индексүүд
        self.param_shapes = [(5, 3), (3,), (2, 4, 2)]
        self.param_indices = [0]
        
        current_idx = 0
        for shape in self.param_shapes:
            num_elements = torch.prod(torch.tensor(shape)).item()
            current_idx += num_elements
            self.param_indices.append(current_idx)
    
    def get_param(self, idx):
        """Тодорхой параметрийг задлах"""
        start = self.param_indices[idx]
        end = self.param_indices[idx + 1]
        shape = self.param_shapes[idx]
        return self.flat_params[start:end].view(shape)
    
    def forward(self, x):
        # Параметрүүдийг задлаад ашиглах
        W1 = self.get_param(0)
        b1 = self.get_param(1)
        W2 = self.get_param(2).view(2, 8)  # 3D-г 2D болгох
        
        h = torch.matmul(x, W1.t()) + b1
        h = torch.relu(h)
        output = torch.matmul(h, W2)
        return output

# Туршилт
model = FlattenedParameters(5*3 + 3 + 2*4*2)
print(f"Нийт параметр: {model.flat_params.numel()}")
for i in range(3):
    param = model.get_param(i)
    print(f"Параметр {i}: {param.shape}")

Нийт параметр: 34
Параметр 0: torch.Size([5, 3])
Параметр 1: torch.Size([3])
Параметр 2: torch.Size([2, 4, 2])


### 2. Параметрийн хязгаарлалт


In [8]:
class ConstrainedModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Хязгаарлагдмал параметрүүд
        self.weight = nn.Parameter(torch.randn(3, 3))
        self.bias = nn.Parameter(torch.zeros(3))
        
        # Проекцийн параметр (orthogonal байх)
        self.orthogonal_weight = nn.Parameter(torch.randn(4, 4))
    
    def apply_constraints(self):
        """Параметрүүдэд хязгаарлалт хэрэгжүүлэх"""
        with torch.no_grad():
            # Weight-ийг нормчлох
            self.weight.data = self.weight.data / self.weight.data.norm(dim=1, keepdim=True)
            
            # Bias-ийг хязгаарлах
            self.bias.data = torch.clamp(self.bias.data, min=-1, max=1)
            
            # Ортогональ жин (QR задрал)
            q, r = torch.linalg.qr(self.orthogonal_weight.data)
            self.orthogonal_weight.data = q
    
    def forward(self, x):
        # Суралцах явцад хязгаарлалт хэрэгжүүлэх
        self.apply_constraints()
        return torch.matmul(x, self.weight) + self.bias

model = ConstrainedModel()

# Суралцахын өмнө
print("Анхны параметрүүд:")
for name, param in model.named_parameters():
    print(f"{name}: норм = {param.data.norm():.4f}")

# Хязгаарлалт хэрэгжүүлэх
model.apply_constraints()

print("\nХязгаарлалт хэрэгжсэний дараа:")
for name, param in model.named_parameters():
    print(f"{name}: норм = {param.data.norm():.4f}")

Анхны параметрүүд:
weight: норм = 4.8430
bias: норм = 0.0000
orthogonal_weight: норм = 3.8193

Хязгаарлалт хэрэгжсэний дараа:
weight: норм = 1.7321
bias: норм = 0.0000
orthogonal_weight: норм = 2.0000


### 3. Параметр үзэмж (property)

In [9]:
class ParameterProperty(nn.Module):
    def __init__(self):
        super().__init__()
        self._weight = None
        self._bias = None
    
    @property
    def weight(self):
        """Жингийн параметрийг үзэмжээр буцаах"""
        if self._weight is None:
            # Хэрэгтэй үед л үүсгэх
            self._weight = nn.Parameter(torch.randn(3, 3))
        return self._weight
    
    @property
    def bias(self):
        """Хазайлтын параметрийг үзэмжээр буцаах"""
        if self._bias is None:
            self._bias = nn.Parameter(torch.zeros(3))
        return self._bias
    
    def forward(self, x):
        # Үзэмжүүдийг ашиглах
        return torch.matmul(x, self.weight) + self.bias

model = ParameterProperty()

# named_parameters() ажиллахгүй, учир нь Parameter-ууд нь attributes биш
print(f"Параметрүүдийн тоо: {len(list(model.parameters()))}")  # 0

# Гэхдээ forward() ажиллана
x = torch.randn(2, 3)
output = model(x)
print(f"Гаралтын хэлбэр: {output.shape}")

Параметрүүдийн тоо: 0
Гаралтын хэлбэр: torch.Size([2, 3])


### 4. Параметрийн хуваалт (Shared Parameter)

In [10]:
class SharedParameterModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Хуваалцсан параметр
        self.shared_weight = nn.Parameter(torch.randn(5, 5))
        
        # Тусдаа параметрүүд
        self.layer1_bias = nn.Parameter(torch.zeros(5))
        self.layer2_bias = nn.Parameter(torch.zeros(5))
    
    def forward(self, x):
        # Нэг жингийг олон удаа ашиглах
        h1 = torch.matmul(x, self.shared_weight.t()) + self.layer1_bias
        h1 = torch.relu(h1)
        
        h2 = torch.matmul(h1, self.shared_weight.t()) + self.layer2_bias
        h2 = torch.relu(h2)
        
        return h2

model = SharedParameterModel()

# Параметрүүдийн тоо хязгаарлагдмал
print("Параметрүүд:")
for name, param in model.named_parameters():
    print(f"  {name}: {param.shape}")

print(f"\nНийт параметр: {sum(p.numel() for p in model.parameters())}")

Параметрүүд:
  shared_weight: torch.Size([5, 5])
  layer1_bias: torch.Size([5])
  layer2_bias: torch.Size([5])

Нийт параметр: 35


## Бусад чухал асуудлууд
### 1. Параметрийг хуулах, шилжүүлэх

In [11]:
# Параметр үүсгэх
param = nn.Parameter(torch.randn(3, 3))

# Хуулах
param_clone = param.clone()  # Зөвхөн утгыг хуулна, Parameter биш
param_copy = nn.Parameter(param.data.clone())  # Шинэ Parameter үүсгэх

print(f"Анхны: {type(param)}, requires_grad: {param.requires_grad}")
print(f"Клон: {type(param_clone)}, requires_grad: {param_clone.requires_grad}")
print(f"Хуулбар: {type(param_copy)}, requires_grad: {param_copy.requires_grad}")

# Төхөөрөмж шилжүүлэх
if torch.cuda.is_available():
    param_gpu = param.cuda()
    print(f"\nТөхөөрөмж: CPU={param.device}, GPU={param_gpu.device}")

Анхны: <class 'torch.nn.parameter.Parameter'>, requires_grad: True
Клон: <class 'torch.Tensor'>, requires_grad: True
Хуулбар: <class 'torch.nn.parameter.Parameter'>, requires_grad: True


### 2. state_dict-д оролцоо

In [12]:
class StateDictModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.param1 = nn.Parameter(torch.randn(2, 3))
        self.param2 = nn.Parameter(torch.zeros(5))
        self.buffer = torch.tensor([1, 2, 3])  # Buffer, Parameter биш
    
    def forward(self, x):
        return x + self.param2.mean()

model = StateDictModel()

# State dict авах
state_dict = model.state_dict()
print("State dict түлхүүрүүд:")
for key in state_dict.keys():
    print(f"  {key}")

# Хадгалах, дуурайх
torch.save(state_dict, 'model_params.pth')

# Шинэ модельд дуурайх
new_model = StateDictModel()
new_model.load_state_dict(torch.load('model_params.pth'))

State dict түлхүүрүүд:
  param1
  param2


<All keys matched successfully>

### 3. Параметр дагах

In [13]:
# Параметрийн өөрчлөлтийг дагах
param = nn.Parameter(torch.tensor(1.0))

# Хуучин утгыг хадгалах
old_value = param.data.clone()

# Параметр өөрчлөх
param.data += 1.0

# Өөрчлөлтийг хэмжих
change = torch.abs(param.data - old_value)
print(f"Өөрчлөлт: {change.item()}")

Өөрчлөлт: 1.0


## Алдаа гаргах тохиолдлууд

### 1. Tensor-ыг Parameter болгохгүй алдаа

In [14]:
class WrongModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.weight = torch.randn(3, 3)  # Алдаа: nn.Parameter биш
    
    def forward(self, x):
        return torch.matmul(x, self.weight)

model = WrongModel()

try:
    # Параметр байхгүй учраас суралцах боломжгүй
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    print("Оптимизатор амжилттай үүслээ")
except ValueError as e:
    print(f"Алдаа: {e}")

Алдаа: optimizer got an empty parameter list


### 2. requires_grad мэдрэмтгий байдал

In [15]:
# Градиент тооцохгүй Parameter
param = nn.Parameter(torch.tensor(1.0), requires_grad=False)

# Алдаа: Градиент шаардсан үйлдэл
try:
    loss = param * 2
    loss.backward()  # Алдаа: param.requires_grad = False
    print("Амжилттай")
except RuntimeError as e:
    print(f"Алдаа: {e}")

# Шийдэл: requires_grad-ыг True болгох
param.requires_grad = True
loss = param * 2
loss.backward()
print(f"Градиент: {param.grad}")

Алдаа: element 0 of tensors does not require grad and does not have a grad_fn
Градиент: 2.0


## Практик зөвлөмж
1. Нэрийн конвенц: Параметрүүдэд тодорхой нэр өгөх

2. Эхлэх утга: Параметрүүдийг сайн эхлэх утгаар эхлүүлэх

3. requires_grad: Зөвхөн шаардлагатай параметрүүдэд градиент идэвхжүүлэх

4. Хязгаарлалт: Шаардлагатай бол параметрүүдэд хязгаарлалт хэрэгжүүлэх

5. Хуваалцсан параметр: Ижил жинг олон газар ашиглах

## Дүгнэлт
- nn.Parameter нь:

1. Суралцах боломж: Автоматаар градиент тооцоонд орно

2. Модуль интеграци: nn.Module-той сайн ажиллана

3. Уян хатан: Tensor-ийн бүх давуу талыг агуулна

4. Удирдах боломж: Хялбар удирдах, хадгалах боломжтой

    Энэ нь PyTorch-ийн суралцах параметрүүдийг тодорхойлох стандарт арга бөгөөд бүх нейрон сүлжээний загварт чухал үүрэгтэй.